In [1]:
import pandas as pd
import numpy as np
import sys
from scipy.sparse import *
from scipy import *
import pickle

In [2]:
# read training set
training = pd.read_csv("training.csv")[['user', 'track']]
training.head()

,user,track
0,000c556aa021426138c47076d53dfbec,spotify:track:4t1iATlAmtRUidtyJkX83E
1,000c556aa021426138c47076d53dfbec,spotify:track:6pb5BBnIM5IM7R1cqag6rE
2,000c556aa021426138c47076d53dfbec,spotify:track:4H9637mkUDyk9Rq0WgDEwc
3,000c556aa021426138c47076d53dfbec,spotify:track:2frPJPY60CPvMwESF6rEXk
4,000c556aa021426138c47076d53dfbec,spotify:track:5ikdUUm6JbnEVnp35c7dvy


In [12]:
tracks = training['track'].unique()
tracks_size = len(tracks)

users = training['user'].unique()
users_size = len(users)


#items = np.zeros((size,size))
users_sparse = csr_matrix( (users_size,tracks_size), dtype=int16 )


print(users_sparse.shape)
      


track_id_dict = {}
id_track_dict = {}
for i,track in enumerate(tracks):
    track_id_dict[track] = i
    id_track_dict[i] = track
    

user_id_dict = {}
id_user_dict = {}
for i,user in enumerate(users):
    user_id_dict[user] = i
    id_user_dict[i] = user
    
    
with open('track_to_id.pickle', 'wb') as handle:
    pickle.dump(track_id_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)    
with open('id_to_track.pickle', 'wb') as handle:
    pickle.dump(id_track_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)     
    
with open('user_to_id.pickle', 'wb') as handle:
    pickle.dump(user_id_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)    
with open('id_to_user.pickle', 'wb') as handle:
    pickle.dump(id_user_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)     

(569, 476357)


In [10]:
# Create user dict
user_dict = {}
groups = training.groupby('user')
for i, group in groups:
    user_id = group['user'].iloc[0]
    track_ids = set(group['track'])
    user_dict[user_id] = track_ids    

In [15]:
# Generate user item matrix
count = 0
for user, tracks in user_dict.items():
    track_ids = [track_id_dict[track] for track in tracks]
    user_id = user_id_dict[user]
    sys.stdout.write("User: "+str(count)+"\r")
    sys.stdout.flush()
    count += 1
    if len(track_ids) < 1500 and len(track_ids) > 25:
        for track_id in track_ids:
            #print(track_id_dict[track_id])
            row = array([user_id for i in track_ids])
            col = array(track_ids)
            data = array([1 for i in track_ids])
            acc = csr_matrix( (data,(row,col)), shape=(users_size,tracks_size), dtype=int16 )
            users_sparse += acc

In [16]:
save_npz('users_sparse.npz', users_sparse)